In [30]:
import msgpack
import os
import numpy as np
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem.rdchem import ChiralType
from rdkit.Chem import AllChem

from torch.utils import data
import random
import random
import torch
import torch.nn.functional as F
from torch_scatter import scatter
from torch_geometric.data import Dataset, Data, DataLoader
import torch_geometric.transforms as T

import math
import gc
from e3nn import o3
import torch
from torch import nn
from torch.nn import functional as F
from torch_cluster import radius, radius_graph
from torch_scatter import scatter
import numpy as np
from e3nn.nn import BatchNorm

In [3]:
torch.cuda.empty_cache()
gc.collect()

0

In [20]:
!pip install e3nn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.7 MB/s eta 0:00:00


In [4]:
!pip install torch-sparse -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.4.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=2809239 sha256=fac618090cb7cafdfc041ac95a83a1cf68c9dff0903b94359aecc7472b5b0cad
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built torch-sparse
Looking in links: https://data.pyg.org/whl/torch-2.4.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-cluster: filename=torch_cluster-1.6.3-cp310-cp310-linux_x86_64.whl size=2039723 sha256=6c72f509a60842637a45c2c950013a1c57b761e764e62a0f8943681fa740ed86
  Stored in directory: /root/.cache/pip/wheels/51/78/c3/536637b3cdcc3313aa5e8851a6c72b97f6a01877e68c7595e3
Successfully built tor

In [3]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html

Looking in links: https://data.pyg.org/whl/torch-2.4.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3658841 sha256=f19d3358bf5ff8e0cea3d92006ec3dda4887000f03373d61362e5d8d6aac9229
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter


In [2]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 50.1 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
direc = "/content/gdrive/MyDrive/GEOM/qm9_crude.msgpack"

qm9_file = os.path.join(direc, "qm9_crude.msgpack")
unpacker = msgpack.Unpacker(open(qm9_file, "rb"))

In [6]:
torch.cuda.empty_cache()
gc.collect()

0

In [7]:
qm9_1k = next(iter(unpacker))

In [8]:
torch.cuda.empty_cache()
gc.collect()

0

In [9]:
sample_smiles = list(qm9_1k.keys())[0]
sample_sub_dic = qm9_1k[sample_smiles]
print(sample_smiles)
{key: val for key, val in sample_sub_dic.items()}

C[C@]12OC[C@H]1[C@H]1OC[C@H]12


{'conformers': [{'xyz': [[6.0, -2.2317852085, -1.0617316596, -0.142905964],
    [6.0, -0.934769739, -0.3116257191, 0.0200955172],
    [6.0, -0.2980379182, 0.610417671, -1.0399619622],
    [1.0, -0.7266087173, 0.8156327908, -2.0152719826],
    [6.0, -0.4000134497, 1.6835934111, 0.0430986576],
    [1.0, 0.5519155098, 2.0880331814, 0.3936706986],
    [1.0, -1.0938571697, 2.4962617908, -0.1864247631],
    [8.0, -0.9820536303, 0.7578886609, 0.9710812471],
    [6.0, 0.994196972, -0.2405163185, -0.9955746708],
    [1.0, 1.3977906131, -0.5689865285, -1.9531003804],
    [6.0, 0.3712050413, -1.1536503187, 0.0696675486],
    [1.0, 0.2670309017, -2.2299344587, -0.0205827014],
    [6.0, 1.4760054102, -0.6042165882, 0.9732408497],
    [1.0, 1.1345298091, -0.0147291083, 1.8268110693],
    [1.0, 2.2123078401, -1.344654808, 1.2952112105],
    [8.0, 1.996285691, 0.2080099819, -0.0906327899],
    [1.0, -2.4509267493, -1.6128426296, 0.7687197358],
    [1.0, -2.1563225275, -1.7564294196, -0.9757184239],
  

In [10]:
!pip install e3fp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.7/713.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.3 MB/s eta 0:00:00


In [10]:
from IPython.display import display

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolDescriptors import CalcWHIM
from rdkit.Chem.rdmolfiles import MolToPDBFile
from rdkit.Chem.Draw import MolsToGridImage

from e3fp.pipeline import fprints_from_mol

from matplotlib import pyplot as plt
import numpy as np

confs = sample_sub_dic["conformers"]

for conf in confs:
    #mol = conf['rd_mol']
    weight = conf["boltzmannweight"]
    degen = conf["degeneracy"]
    en = conf["relativeenergy"]

    print(f"Boltzmann weight: {weight}")
    print(f"Degeneracy: {degen}")
    print(f"Relative energy: {en}")
    #display(MolsToGridImage([mol]))

Boltzmann weight: 1.0
Degeneracy: 3
Relative energy: 0.0


In [11]:
chirality = {ChiralType.CHI_TETRAHEDRAL_CW: -1.,
             ChiralType.CHI_TETRAHEDRAL_CCW: 1.,
             ChiralType.CHI_UNSPECIFIED: 0,
             ChiralType.CHI_OTHER: 0}

In [12]:
bonds = {BT.SINGLE: 0, BT.DOUBLE: 1, BT.TRIPLE: 2, BT.AROMATIC: 3}
qm9_types = {'H': 0, 'C': 1, 'N': 2, 'O': 3, 'F': 4}
drugs_types = {'H': 0, 'Li': 1, 'B': 2, 'C': 3, 'N': 4, 'O': 5, 'F': 6, 'Na': 7, 'Mg': 8, 'Al': 9, 'Si': 10,
               'P': 11, 'S': 12, 'Cl': 13, 'K': 14, 'Ca': 15, 'V': 16, 'Cr': 17, 'Mn': 18, 'Cu': 19, 'Zn': 20,
               'Ga': 21, 'Ge': 22, 'As': 23, 'Se': 24, 'Br': 25, 'Ag': 26, 'In': 27, 'Sb': 28, 'I': 29, 'Gd': 30,
               'Pt': 31, 'Au': 32, 'Hg': 33, 'Bi': 34}

In [13]:
def one_k_encoding(value, choices):
    """
    Creates a one-hot encoding with an extra category for uncommon values.
    :param value: The value for which the encoding should be one.
    :param choices: A list of possible values.
    :return: A one-hot encoding of the :code:`value` in a list of length :code:`len(choices) + 1`.
             If :code:`value` is not in :code:`choices`, then the final element in the encoding is 1.
    """
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1

    return encoding

In [43]:
def featurize_mol(mol, types=qm9_types):
  """
  Featurisation code taken from GeoMol https://github.com/gcorso/torsional-diffusion
    Returns:
        x:  node features
        z: atomic numbers of the nodes (the symbol one hot is included in x)
        edge_index: [2, E] tensor of node indices forming edges
        edge_attr: edge features
    """
  if type(types) is str:
    if types == 'qm9':
      types = qm9_types
    elif types == 'drugs':
      types = drugs_types

  N =  mol.GetNumAtoms()  # Number of atoms in the molecule
  atom_type_idx =[]       # Numerical index for each atom's type
  atomic_number =[]       # Atomic number of each atom
  atom_features = []
  chiral_tag = []
  ring = mol.GetRingInfo()   # Information about whether atoms are part of a ring structure
  for i, atom in enumerate(mol.GetAtoms()):
    atom_type_idx.append(types[atom.GetSymbol()])
    chiral_tag.append(chirality[atom.GetChiralTag()])
    atomic_number.append(atom.GetAtomicNum())
    atom_features.extend([atom.GetAtomicNum(),
                            1 if atom.GetIsAromatic() else 0])  # Adds atomic number and whether the atom is aromatic (part of a ring structure).
    atom_features.extend(one_k_encoding(atom.GetDegree(), [0,1,2,3,4,5,6])) # One-hot encoding of atom_degrees
    atom_features.extend(one_k_encoding(atom.GetHybridization(),[
        Chem.rdchem.HybridizationType.SP,
        Chem.rdchem.HybridizationType.SP2,
        Chem.rdchem.HybridizationType.SP3,
        Chem.rdchem.HybridizationType.SP3D,
        Chem.rdchem.HybridizationType.SP3D2]))
    atom_features.extend(one_k_encoding(atom.GetImplicitValence(),[0, 1, 2, 3, 4, 5, 6]))
    atom_features.extend(one_k_encoding(atom.GetFormalCharge(),[-1, 0, 1]))
    atom_features.extend([int(ring.IsAtomInRingOfSize(i,3)),
                          int(ring.IsAtomInRingOfSize(i,4)),
                          int(ring.IsAtomInRingOfSize(i, 5)),
                          int(ring.IsAtomInRingOfSize(i, 6)),
                          int(ring.IsAtomInRingOfSize(i, 7)),
                          int(ring.IsAtomInRingOfSize(i, 8))])
    atom_features.extend(one_k_encoding(ring.NumAtomRings(i),[0, 1, 2, 3]))
  z = torch.tensor(atomic_number, dtype=torch.long)

  row,col,edge_type = [], [], []
  for bond in mol.GetBonds():  # gives all the bonds in the molecule and for each bond
    start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()   # Returns ID of the atom where the bond starts and where the bond ends
    row += [start, end]
    col += [end,start]
    edge_type += 2* [bonds[bond.GetBondType()]]
  edge_index = torch.tensor([row,col], dtype = torch.long)
  edge_type = torch.tensor(edge_type, dtype = torch.long)
  edge_attr = F.one_hot(edge_type, num_classes = len(bonds)).to(torch.float)

  x1= F.one_hot(torch.tensor(atom_type_idx), num_classes = len(types))
  x2 = torch.tensor(atom_features).view(N, -1)
  x = torch.cat([x1.to(torch.float), x2], dim=-1)

  #conf = mol.GetConformer()
  #pos = torch.tensor([list(conf.GetAtomPosition(i)) for i in range(mol.GetNumAtoms())], dtype=torch.float)

  return Data(x=x, edge_index = edge_index, edge_attr= edge_attr, z=z)


def featurize_mol_from_smiles(smiles, dataset='qm9'):
    if dataset == 'qm9':
        types = qm9_types
    elif dataset == 'drugs' or dataset == 'bace':
        types = drugs_types

    # Filter fragments
    if '.' in smiles:
        print(f"Skipping fragment: {smiles}")
        return None, None

    # Filter mols RDKit can't handle
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        mol = Chem.AddHs(mol)
        #if AllChem.EmbedMolecule(mol) == -1:
            #print(f"3D embedding failed for: {smiles}")
            #return None, None
        #AllChem.UFFOptimizeMolecule(mol)
    else:
        print(f"Invalid SMILES: {smiles}")
        return None, None

    N = mol.GetNumAtoms()
    print(f"Num atoms: {N}")

    # Filter out small molecules
    if N < 4:
        print(f"Skipping small molecule: {smiles}")
        return None, None

    data = featurize_mol(mol, types)
    data.name = smiles
    return mol, data


In [15]:
class BatchSampler(data.Sampler):
    '''
    From https://github.com/jingraham/neurips19-graph-protein-design.

    A `torch.utils.data.Sampler` which samples batches according to a maximum number of graph nodes.

    :param node_counts: array of node counts in the dataset to sample from
    :param max_nodes: the maximum number of nodes in any batch,
                      including batches of a single element
    :param shuffle: if `True`, batches in shuffled order
    '''
    def __init__(self, node_counts, max_nodes=5000, shuffle=True):

        self.node_counts = node_counts
        self.idx = [i for i in range(len(node_counts))
                        if node_counts[i] <= max_nodes]
        self.shuffle = shuffle
        self.max_nodes = max_nodes
        self._form_batches()

    def _form_batches(self):
        self.batches = []
        if self.shuffle: random.shuffle(self.idx)
        idx = self.idx
        while idx:
            batch = []
            n_nodes = 0
            while idx and n_nodes + self.node_counts[idx[0]] <= self.max_nodes:
                next_idx, idx = idx[0], idx[1:]
                n_nodes += self.node_counts[next_idx]
                batch.append(next_idx)
            self.batches.append(batch)

    def __len__(self):
        if not self.batches: self._form_batches()
        return len(self.batches)

    def __iter__(self):
        if not self.batches: self._form_batches()
        for batch in self.batches: yield batch

In [44]:
class MoleculeDataset(Dataset):
    def __init__(self, smiles_list, dataset_type='qm9', transform=None, pre_transform=None):
        self.smiles_list = []
        self.node_counts = []
        self.dataset_type = dataset_type
        super(MoleculeDataset, self).__init__(None, transform, pre_transform)

        # Filter invalid SMILES and store only valid molecules
        for smiles in smiles_list:
            mol, data = featurize_mol_from_smiles(smiles, dataset=self.dataset_type)
            if data is not None:
                self.smiles_list.append(smiles)  # Only store valid SMILES
                self.node_counts.append(data.num_nodes)  # Store node count
            else:
                print(f"Skipping invalid SMILES: {smiles}")

    def len(self):
        return len(self.smiles_list)

    def get(self, idx):
        smiles = self.smiles_list[idx]
        mol, data = featurize_mol_from_smiles(smiles, dataset=self.dataset_type)
        return data

In [45]:
sample_smiles = list(qm9_1k.keys())
valid_smiles = [smiles for smiles in sample_smiles if smiles is not None]
transform = T.AddRandomWalkPE(walk_length=20, attr_name='pe')
dataset = MoleculeDataset(smiles_list = valid_smiles, dataset_type='qm9',pre_transform = transform)
batch_sampler = BatchSampler(node_counts=dataset.node_counts, max_nodes=2000, shuffle=True)

data_loader = DataLoader(dataset, batch_sampler=batch_sampler)

for batch in data_loader:
    if batch is None:
        continue
    print(batch)

Num atoms: 19
Num atoms: 15
Num atoms: 21
Num atoms: 20
Num atoms: 15
Num atoms: 22
Num atoms: 21
Num atoms: 20
Num atoms: 18
Num atoms: 21
Num atoms: 20
Num atoms: 16
Num atoms: 20
Num atoms: 23
Num atoms: 17
Num atoms: 19
Num atoms: 16
Num atoms: 18
Num atoms: 18
Num atoms: 15
Num atoms: 15
Num atoms: 16
Num atoms: 17
Num atoms: 19
Num atoms: 19
Num atoms: 18
Num atoms: 21
Num atoms: 20
Num atoms: 16
Num atoms: 16
Num atoms: 16
Num atoms: 17
Num atoms: 19
Num atoms: 13
Num atoms: 13
Num atoms: 17
Num atoms: 23
Num atoms: 20
Num atoms: 17
Num atoms: 20
Num atoms: 21
Num atoms: 16
Num atoms: 23
Num atoms: 19
Num atoms: 20
Num atoms: 21
Num atoms: 23
Num atoms: 17
Num atoms: 19
Num atoms: 21
Num atoms: 21
Num atoms: 23
Num atoms: 14
Num atoms: 23
Num atoms: 20
Num atoms: 20
Num atoms: 15
Num atoms: 19
Num atoms: 16
Num atoms: 21
Num atoms: 16
Num atoms: 12
Num atoms: 23
Num atoms: 20
Num atoms: 17
Num atoms: 14
Num atoms: 23
Num atoms: 21
Num atoms: 16
Num atoms: 19
Num atoms: 21
Num at

[23:44:34] Can't kekulize mol.  Unkekulized atoms: 2 4 5
[23:44:34] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 7


Num atoms: 17
Num atoms: 23
Num atoms: 14
Num atoms: 15
Num atoms: 18
Num atoms: 23
Num atoms: 27
Num atoms: 16
Num atoms: 14
Num atoms: 22
Num atoms: 18
Num atoms: 14
Num atoms: 19
Num atoms: 13
Num atoms: 14
Num atoms: 17
Num atoms: 17
Num atoms: 17
Num atoms: 19
Num atoms: 21
Num atoms: 24
Num atoms: 20
Num atoms: 19
Num atoms: 18
Num atoms: 23
Num atoms: 16
Num atoms: 22
Num atoms: 18
Num atoms: 12
Num atoms: 11
Num atoms: 18
Num atoms: 16
Num atoms: 18
Num atoms: 21
Num atoms: 25
Num atoms: 16
Num atoms: 18
Num atoms: 18
Num atoms: 23
Num atoms: 17
Num atoms: 23
Num atoms: 25
Num atoms: 11
Num atoms: 17
Num atoms: 18
Num atoms: 25
Num atoms: 21
Num atoms: 21
Num atoms: 13
Num atoms: 16
Num atoms: 16
Num atoms: 14
Num atoms: 19
Num atoms: 12
Num atoms: 12
Invalid SMILES: COc1cnn[nH]c1[NH]
Skipping invalid SMILES: COc1cnn[nH]c1[NH]
Num atoms: 19
Num atoms: 20
Num atoms: 14
Num atoms: 17
Num atoms: 19
Num atoms: 17
Num atoms: 13
Num atoms: 13
Num atoms: 19
Num atoms: 21
Num atoms: 18

[23:44:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[23:44:35] Explicit valence for atom # 0 N, 4, is greater than permitted
[23:44:35] Explicit valence for atom # 2 N, 4, is greater than permitted
[23:44:35] Explicit valence for atom # 0 N, 4, is greater than permitted


Num atoms: 15
Num atoms: 17
Num atoms: 23
Num atoms: 13
Num atoms: 20
Num atoms: 12
Num atoms: 17
Num atoms: 17
Num atoms: 19
Num atoms: 19
Num atoms: 18
Num atoms: 16
Num atoms: 15
Num atoms: 16
Num atoms: 13
Num atoms: 17
Num atoms: 15
Num atoms: 16
Num atoms: 19
Num atoms: 17
Num atoms: 12
Num atoms: 16
Num atoms: 15
Num atoms: 18
Num atoms: 21
Num atoms: 14
Num atoms: 17
Num atoms: 14
Num atoms: 17
Num atoms: 14
Num atoms: 17
Num atoms: 21
Num atoms: 21
Invalid SMILES: NC(=[NH2])N1[CH]C(=O)C=N1
Skipping invalid SMILES: NC(=[NH2])N1[CH]C(=O)C=N1
Num atoms: 17
Num atoms: 18
Num atoms: 18
Num atoms: 21
Num atoms: 17
Invalid SMILES: [NH2]=C(N)[C](C#N)C#N
Skipping invalid SMILES: [NH2]=C(N)[C](C#N)C#N
Num atoms: 16
Num atoms: 19
Num atoms: 15
Num atoms: 16
Num atoms: 15
Num atoms: 21
Num atoms: 17
Num atoms: 16
Num atoms: 19
Num atoms: 16
Num atoms: 15
Num atoms: 19
Num atoms: 18
Num atoms: 19
Num atoms: 15
Num atoms: 20
Num atoms: 19
Num atoms: 18
Num atoms: 19
Num atoms: 12
Num atoms:

[23:44:35] Can't kekulize mol.  Unkekulized atoms: 1 2 3 6 7



Num atoms: 18
Num atoms: 19
Num atoms: 20
Num atoms: 14
Num atoms: 14
Num atoms: 20
Num atoms: 21
Num atoms: 18
Num atoms: 23
Num atoms: 18
Num atoms: 18
Num atoms: 18
Num atoms: 21
Num atoms: 19
Num atoms: 14
Num atoms: 17
Num atoms: 18
Num atoms: 19
Num atoms: 23
Num atoms: 18
Num atoms: 15
Num atoms: 19
Num atoms: 13
Num atoms: 21
Num atoms: 18
Num atoms: 9
Num atoms: 15
Num atoms: 22
Num atoms: 18
Num atoms: 17
Num atoms: 22
Num atoms: 16
Num atoms: 18
Num atoms: 20
Num atoms: 22
Num atoms: 18
Num atoms: 19
Num atoms: 19
Num atoms: 19
Num atoms: 9
Num atoms: 16
Num atoms: 19
Num atoms: 21
Num atoms: 20
Num atoms: 21
Num atoms: 15
Num atoms: 14
Num atoms: 21
Num atoms: 16
Num atoms: 19
Num atoms: 19
Num atoms: 18
Num atoms: 12
Num atoms: 17
Num atoms: 14
Num atoms: 16
Num atoms: 15
Num atoms: 18
Num atoms: 16
Num atoms: 19
Num atoms: 18
Num atoms: 23
Num atoms: 19
Num atoms: 27
Num atoms: 15
Num atoms: 15
Num atoms: 16
Num atoms: 16
Num atoms: 18
Num atoms: 19
Num atoms: 14
Num ato

[23:44:36] Explicit valence for atom # 2 N, 4, is greater than permitted
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Num atoms: 18
Num atoms: 15
Num atoms: 22
Num atoms: 18
Num atoms: 25
Num atoms: 17
Num atoms: 14
Num atoms: 13
Num atoms: 20
Num atoms: 23
Num atoms: 23
Num atoms: 17
Num atoms: 21
Num atoms: 19
Num atoms: 21
Num atoms: 18
Num atoms: 23
Num atoms: 18
Num atoms: 15
Num atoms: 13
Num atoms: 18
Num atoms: 16
Num atoms: 19
Num atoms: 21
Num atoms: 16
Num atoms: 18
Num atoms: 23
Num atoms: 17
Num atoms: 20
Num atoms: 23
Num atoms: 17
Num atoms: 18
Num atoms: 13
Num atoms: 19
Num atoms: 20
Num atoms: 18
Num atoms: 21
Num atoms: 22
Num atoms: 16
Num atoms: 19
Num atoms: 12
Num atoms: 15
DataBatch(x=[1994, 44], edge_index=[2, 4140], edge_attr=[4140, 4], z=[1994], name=[110], batch=[1994], ptr=[111])
Num atoms: 16
Num atoms: 15
Num atoms: 19
Num atoms: 21
Num atoms: 20
Num atoms: 17
Num atoms: 19
Num atoms: 18
Num atoms: 15
Num atoms: 21
Num atoms: 19
Num atoms: 18
Num atoms: 14
Num atoms: 18
Num atoms: 14
Num atoms: 17
Num atoms: 12
Num atoms: 15
Num atoms: 15
Num atoms: 17
Num atoms: 18
Num 

In [19]:
torch.cuda.empty_cache()
gc.collect()

0

In [20]:
class TensorProductConvLayer(torch.nn.Module):

  "Params: node-attr: (num_nodes, node_features), edge_index : (2,num_edges), edge_attr: (num_edges, n_edge_features), edge_sh, out_nodes"

  def __init__(self, in_irreps, sh_irreps, out_irreps, n_edge_features, residual= True, batch_norm = True):
    super(TensorProductConvLayer,self).__init__()
    self.in_irreps = in_irreps
    self.out_irreps = out_irreps
    self.sh_irreps = sh_irreps

    self.tp = tp = o3.FullyConnectedTensorProduct(in_irreps, sh_irreps, out_irreps, shared_weights = False)
    self.fc = nn.Sequential(nn.Linear(
        n_edge_features, n_edge_features),
         nn.ReLU(),
         nn.Linear(n_edge_features, tp.weight_numel))

    self.batch_norm = BatchNorm(out_irreps) if batch_norm else None

  def forward(self, node_attr, edge_index, edge_attr, edge_sh, out_nodes= None, reduce='mean'):

    "Params: node_attr[edge_dst]: node attributes from destination nodes of the edges, edge_sh, self.fc(edge_attr)"

    edge_src, edge_dst = edge_index
    tp = self.tp(node_attr[edge_dst], edge_sh,self.fc(edge_attr))

    out_nodes = out_nodes or node_attr.shape[0]
    out = scatter(tp, edge_src, dim=0, dim_size = out_nodes, reduce=reduce)
    if self.residual:
      padded = F.pad(node_attr, (0, out.shape[-1] - node_attr.shape[-1]))
      out = out + padded

    if self.batch_norm:
      out = self.batch_norm(out)

    return out

In [21]:
class GaussianSmearing(torch.nn.Module):
    def __init__(self, start=0.0, stop=5.0, num_gaussians=50):
        super().__init__()
        offset = torch.linspace(start, stop, num_gaussians)
        self.coeff = -0.5 / (offset[1] - offset[0]).item() ** 2
        self.register_buffer('offset', offset)

    def forward(self, dist):
        dist = dist.view(-1, 1) - self.offset.view(1, -1)
        return torch.exp(self.coeff * torch.pow(dist, 2))

In [22]:
class TensorProductEquivariantModel(torch.nn.Module):

  def __init__(self, in_node_features = 44, in_edge_features = 4,sh_lmax = 2, ns= 32, nv = 8, num_conv_layers = 4, max_radius = 5, radius_embed_dim =50,
             use_second_order_repr= True, batch_norm = True, residual = True):

    "This code is taken from GeoMol https://github.com/gcorso/torsional-diffusion"

    super(TensorProductEquivariantModel, self).__init__()
    self.in_node_features = in_node_features
    self.in_edge_features = in_edge_features
    self.max_radius = max_radius
    self.radius_emed_dim = radius_embed_dim
    self.sh_irreps = o3.Irreps.spherical_harmonics(lmax = sh_lmax)
    self.ns , self.nv = ns, nv

    self.node_embedding = nn.Sequential(
        nn.Linear(in_node_features, ns),
        nn.ReLU(),
        nn.Linear(ns, ns),
        nn.ReLU()
    )

    self.edge_embedding = nn.Sequential(
        nn.Linear(in_edge_features + radius_embed_dim, ns),
        nn.ReLU(),
        nn.Linear(ns, ns),
        nn.ReLU())

    self.distance_expansion = GaussianSmearing(0.0, max_radius, radius_embed_dim)
    conv_layers = []

    if use_second_order_repr:
      irrep_seq = [
          f'{ns}x0e',
          f'{ns}x0e + {nv}x1o + {nv}x2e',
          f'{ns}x0e + {nv}x1o + {nv}x2e + {nv}x1e + {nv}x2o',
          f'{ns}x0e + {nv}x1o + {nv}x2e + {nv}x1e + {nv}x2o + {ns}x0o'
      ]

    else:
            irrep_seq = [
                f'{ns}x0e',
                f'{ns}x0e + {nv}x1o',
                f'{ns}x0e + {nv}x1o + {nv}x1e',
                f'{ns}x0e + {nv}x1o + {nv}x1e + {ns}x0o'
            ]
    for i in range(num_conv_layers):
      in_irreps = irrep_seq[min(i, len(irrep_seq) - 1)]
      out_irreps = irrep_seq[min(i+1, len(irrep_seq)-1)]

      layer = TensorProductConvLayer(
          in_irreps = in_irreps,
          sh_irreps = self.sh_irreps,
          out_irreps = out_irreps,
          n_edge_features = 3 * ns,
          residual = residual,
          batch_norm = batch_norm
      )

      conv_layers.append(layer)

    self.conv_layers = nn.ModuleList(conv_layers)
    self.final_edge_embedding = nn.Sequential(
        nn.Linear(ns, ns),
        nn.ReLU(),
        nn.Linear(ns, ns)
    )

    self.final_tp = o3.FullTensorProduct(self.sh_irreps, "2e")

    self.bond_conv = TensorProductConvLayer(
        in_irreps = self.conv_layers[-1].out_irreps,
        sh_irreps = self.final_tp.irreps_out,
        out_irreps = f'{ns}x0o',
        n_edge_features = 3*ns,
        residual = False,
        batch_norm = batch_norm
    )

    self.final_layer = nn.Sequential(
        nn.Linear(ns,ns, bias = False),
        nn.Tanh(),
        nn.Linear(ns, 1, bias= False)
    )

  def forward(self,data):
      node_attr, edge_index, edge_attr, edge_sh = self.build_conv_graph(data)
      src, dst = edge_index

      node_attr = self.node_embedding(node_attr)
      edge_attr = self.edge_embedding(edge_attr)

      for layer in self.conv_layers:
        edge_attr_ = torch.cat([edge_attr, node_attr[src, :self.ns], node_attr[dst, :self.ns]],-1)
        node_attr = layer(node_attr, edge_index, edge_attr_, edge_sh, reduce ='mean')

      bonds, edge_index, edge_attr, edge_sh = self.build_bond_conv_graph(data, node_attr)

      bond_vec = data.pos[bonds[1]] - data.pos[bonds[0]]
      bond_attr = node_attr[bonds[0]] + node_attr[bonds[1]]

      bonds_sh = o3.spherical_harmonics("2e", bond_vec, normalize = True, normalization = 'component')
      egde_sh = self.final_tp(edge_sh, bonds_sh[edge_index[0]])

      edge_attr = torch.cat([edge_attr, node_attr[edge_index[1], :self.ns], bond_attr[edge_index[0], :self.ns]], -1)
      out = self.bond_conv(node_attr, edge_index, edge_attr, edge_sh, out_nodes= data.edge_mask.sum(), reduce='mean')


      out = self.final_linear(out)

      data.edge_pred = out.squeeze()

      return data

  def build_bond_conv_graph(self, data, node_attr):

        bonds = data.edge_index[:, data.edge_mask].long()
        bond_pos = (data.pos[bonds[0]] + data.pos[bonds[1]]) / 2
        bond_batch = data.batch[bonds[0]]
        edge_index = radius(data.pos, bond_pos, self.max_radius, batch_x=data.batch, batch_y=bond_batch)

        edge_vec = data.pos[edge_index[1]] - bond_pos[edge_index[0]]
        edge_attr = self.distance_expansion(edge_vec.norm(dim=-1))

        edge_attr = self.final_edge_embedding(edge_attr)

        edge_sh = o3.spherical_harmonics(self.sh_irreps, edge_vec, normalize=True, normalization='component')

        return bonds, edge_index, edge_attr, edge_sh

  def build_conv_graph(self, data):

    # Compute radius-based edges
    radius_edges = radius_graph(data.pos, self.max_radius, data.batch)

    # Combine bond edges and radius-based edges
    edge_index = torch.cat([data.edge_index, radius_edges], 1).long()

    # Initialize edge attributes for radius-based edges as zeros
    edge_attr = torch.cat([
        data.edge_attr,
        torch.zeros(radius_edges.shape[-1], self.in_edge_features, device=data.x.device)
    ], 0)

    # Node attributes remain unchanged
    node_attr = data.x

    # Compute edge vectors
    src, dst = edge_index
    edge_vec = data.pos[dst.long()] - data.pos[src.long()]

    # Edge length embedding
    edge_length_emb = self.distance_expansion(edge_vec.norm(dim=-1))

    # Concatenate edge length embedding with existing edge attributes
    edge_attr = torch.cat([edge_attr, edge_length_emb], 1)

    # Compute spherical harmonics for the edge vectors
    edge_sh = o3.spherical_harmonics(self.sh_irreps, edge_vec, normalize=True, normalization='component')

    return node_attr, edge_index, edge_attr, edge_sh


In [23]:
torch.cuda.empty_cache()
gc.collect()

0

In [24]:
model = TensorProductEquivariantModel()
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model)

/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:178: Us

TensorProductEquivariantModel(
  (node_embedding): Sequential(
    (0): Linear(in_features=44, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  )
  (edge_embedding): Sequential(
    (0): Linear(in_features=54, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  )
  (distance_expansion): GaussianSmearing()
  (conv_layers): ModuleList(
    (0): TensorProductConvLayer(
      (tp): FullyConnectedTensorProduct(32x0e x 1x0e+1x1o+1x2e -> 32x0e+8x1o+8x2e | 1536 paths | 1536 weights)
      (fc): Sequential(
        (0): Linear(in_features=96, out_features=96, bias=True)
        (1): ReLU()
        (2): Linear(in_features=96, out_features=1536, bias=True)
      )
      (batch_norm): BatchNorm (32x0e+8x1o+8x2e, eps=1e-05, momentum=0.1)
    )
    (1): TensorProductConvLayer(
      (tp): FullyConnectedTensorProduct(32x0e+8x1o+8x2e x 1x0e+1x1o+1x2e -> 32x0e+8

In [26]:
torch.cuda.empty_cache()
gc.collect()

0